In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import seaborn as sns
%matplotlib inline

**Compile all features together**

In [ ]:
serv_feat_1 = pd.read_csv('D:/Referral/EDA/Features/rfrl_serv_data_red_new.dsv', low_memory=False)
 #dealer master table to map region and zone
dlr_map = pd.read_csv('D:/Referral/EDA/Features/dlr_map.csv', low_memory=False)
region = pd.read_csv('D:/Referral/EDA/Features/region_code.csv', low_memory=False)
region_dict =  pd.Series(region.cmm_code_name.values,index=region.cmm_code).to_dict()
dlr_map = dlr_map.rename(columns={'DORG_DLR_NO': 'dlr_no',
                                 'DORG_RGN_CODE_DEL': 'region',
                                  'DORG_STCD_CODE': 'state'})
dlr_map['region'] = dlr_map.region.map(region_dict)
dlr_map['zone'] = dlr_map['region'].astype(str).str[0]
serv_feat_map = pd.merge(serv_feat_1, dlr_map, how='left', on=['dlr_no'])

serv_feat_map['yearly_freq'] = np.where((serv_feat_map['vehicle_age']<5),
                                       (serv_feat_map['total_srv_count_3yrs']/serv_feat_map['vehicle_age']),
                                      (serv_feat_map['total_srv_count_3yrs']/5))  

**Pre-processing**

In [ ]:
#Drop correlated columns
cols = cluster_feat.columns.tolist()
cols.remove('unique_cust_grp')
cols.remove('model_cd')
serv_feat_map = cluster_feat[cols]
serv_feat_map = pd.DataFrame(StandardScaler().fit_transform(serv_feat_map),columns = serv_feat_map.columns) 
corr_serv = serv_feat_map.corr().abs()
upper = corr_serv.where(np.triu(np.ones(corr_serv.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

serv_feat_map = serv_feat_map.drop(serv_feat_map[to_drop], axis=1)
serv_feat_map.shape

#IQR method
def outliers_iqr(col, df):
    quartile_1 = df[col].quantile(0.25)
    quartile_3 = df[col].quantile(0.25)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    filter_ = (df[col]> upper_bound) & (df[col]< lower_bound)
    return df.loc[filter_]


cols = list(serv_feat_map.select_dtypes(include=[np.number]).columns.values)
for col in cols:
    serv_feat_map = outliers_iqr(col, serv_feat_map)
    
Creta = ['Creta']
i10_i20 = ['Grand i10', 'i20', 'New i20']
i10_santro = ['i10', 'Santro']
eon_ver = ['Eon', 'Verna', 'New Verna', 'Next Gen Verna', 'Xcent']
cm_x = ['i20 Active', 'Accent']
cm_y = ['Getz', 'Elantra', 'Tucosn', 'Santa Fe', 'New Santa Fe (DM)', 'Sonata',
        'Venue', 'E.F.Sonata', 'New Sonata', 'Terracan', 'Neo Elantra']

# one-hot encoding car models                                 
serv_feat_map['Creta'] = np.where(serv_feat_map['model_cd'].isin(Creta), 1, 0)   
serv_feat_map['i10_i20'] = np.where(serv_feat_map['model_cd'].isin(i10_i20), 1, 0)
serv_feat_map['i10_santro'] = np.where(serv_feat_map['model_cd'].isin(i10_santro), 1, 0)
serv_feat_map['eon_ver'] = np.where(serv_feat_map['model_cd'].isin(eon_ver), 1, 0)
serv_feat_map['cm_x'] = np.where(serv_feat_map['model_cd'].isin(cm_x), 1, 0)
serv_feat_map['cm_y'] = np.where(serv_feat_map['model_cd'].isin(cm_y), 1, 0)
serv_feat_map['used_car'] = np.where(serv_feat_map.vin.str.contains('used'), 1, 0)

# one-hot encoding for other columns
col_cat = ['state', 'region', 'dlr_code_grp']
for col in col_cat:
    one_hot = pd.get_dummies(serv_feat_map[col])
    # Drop column as it is now encoded
    serv_feat_map = serv_feat_map.drop(col,axis = 1)
    # Join the encoded serv_feat_map
    serv_feat_map = serv_feat_map.join(one_hot)

**Nearest neighbot calculation and density based clustering**

In [ ]:
#Standardize all columns
df_subset = serv_feat_map.drop(columns=['unique_cust_grp', 'VIN', 'model_cd', 'Target']) #Drop index columns
cluster_data_subset = df_subset
cluster_data_subset = StandardScaler().fit_transform(cluster_data_subset)

#Calculate ideal Nearestneighbor value for epsilon
sns.set()
neigh = NearestNeighbors(n_neighbors=2, n_jobs=-1)
nbrs = neigh.fit(cluster_data)
print ('Done')
distances, indices = nbrs.kneighbors(cluster_data)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)
plt.yticks(np.arange(0, 6, 0.5))
plt.xticks(np.arange(0, 80000, 10000))
plt.ylim(0,6)
plt.xlim(0, 80000)

#Run DBSCAN based on epsilon value derived from above graph
m = DBSCAN(eps=0.4, min_samples=500, n_jobs=-1)
m.fit(cluster_data)
print ('Done')
serv_feat_map['clusters'] = m.labels_
df_subset['clusters'] = m.labels_

#Run t-sne to visualize clustering data
num_features = df_subset.drop('clusters', axis=1).columns.tolist()
x = df_subset.loc[:, num_features].values
y = df_subset.loc[:,['clusters']].values
x = StandardScaler().fit_transform(x)
tsne = TSNE(n_components=2, verbose=1, perplexity=25, n_iter=500)
tsne_results = tsne.fit_transform(x)

cluster_all['tsne-2d-one'] = tsne_results[:,0]
cluster_all['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(10,8))

sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
     hue="clusters_new",
     palette=sns.color_palette("hls", 101),
    data=cluster_all[cluster_all['clusters'].isin(range(-1, 100))],
    legend=False,
  #  color='r'
)
plt.savefig("tsne_clusters_outliers.png")
plt.show()

**RFM analysis**

In [ ]:
#Calculate recency
serv_RFM = pd.read_csv('serv_RFM.dsv', low_memory=False) #serv_RFM data was created by mapping last RO to serv_feat_map
serv_RFM['Recency_value'] = (pd.Timestamp(year=2019, month=8, day=1) - (
    serv_RFM['repm_ro_dtime']))/np.timedelta64(1,'M')

#Scoring for model basis
model_referral = serv_RFM.groupby(['model_cd', 'Target']).size().unstack()
model_referral['Referral_perc'] = (model_referral[1]*100)/(model_referral[1]+model_referral[0])
#Find quantile cut for Referral perc and select model groups accordingly
display(pd.qcut(model_referral['Referral_perc'], q=[0, 0.20, 0.40, 0.60, 0.80, 1.00]))

grp_5 = ['Tucson']
grp_4 = ['Creta', 'Elantra', "New Santa Fe (DM)", 'i20 Active']
grp_3 = ['Accent', 'Eon', 'Getz', 'Grand i10', 'Santro', 'Santa Fe', 'Verna', 'i10', 'i20']
grp_2 = ['Sonata']
serv_RFM['model_score'] = np.where(serv_RFM['model_new'].isin(grp_5), 5,
                                        np.where(serv_RFM['model_new'].isin(grp_4),
                                                 4, np.where(serv_RFM['model_new'].isin(grp_3),
                                                 3, np.where(serv_RFM['model_new'].isin(grp_2),
                                                 2, 1))))

#Scoring frequency and Recency basis
display(pd.qcut(model_referral['yearly_freq'], q=[0, 0.40, 0.80, 1.00]))
display(pd.qcut(model_referral['last_year_freq'], q=[0, 0.40, 0.80, 1.00]))
display(pd.qcut(model_referral['Recency_value'], q=[0, 0.20, 0.40, 0.60, 0.80, 1.00]))

serv_RFM['Recency_score'] = np.where(serv_RFM['Recency_value']<5, 5,
                                        np.where(serv_RFM['Recency_value'].between(5, 10, inclusive=True),
                                                 4, np.where(serv_RFM['Recency_value'].between(11, 15, inclusive=True),
                                                 3, np.where(serv_RFM['Recency_value'].between(16, 20, inclusive=True),
                                                 2, np.where(serv_RFM['Recency_value'].between(21, 26, inclusive=True),
                                                 1, 0)))))
serv_RFM['Yrly_freq_score'] = np.where(serv_RFM['yearly_freq']<0.5, 0,
                                        np.where(serv_RFM['yearly_freq'].between(0.6, 1.4, inclusive=True),
                                                 2.5, 5))
serv_RFM['Last_yr_freq_score'] = np.where(serv_RFM['last_yr_freq']==0, 0,
                                        np.where(serv_RFM['last_yr_freq'].between(1, 1.9, inclusive=True),
                                                 2.5, 5))


serv_RFM['Final_score'] = 5*(serv_RFM['Recency_score']+serv_RFM['Yrly_freq_score']+serv_RFM['Last_yr_freq_score']+serv_RFM['model_score'])

**Add phone_score and referral score to Final_score**

In [ ]:
RFM_output = serv_RFM[serv_RFM['vin'].isin(cluster_RFM['vin'].tolist())][['unique_cust_grp',
                                                                          'vin', 'repm_dlr_no',
                                                                          'repm_cust_no',
                                                                         'repm_ro_dtime',
                                                                         'Final_score']]
#Serv_count score
serv_count = RFM_output[RFM_output['csph_cust_phone_no'].notnull()].groupby(
    ['unique_cust_grp', 'vin', 'csph_cust_phone_no'])['repm_ro_dtime'].count().reset_index().rename(
    columns={'repm_ro_dtime': 'serv_count'})
serv_count['Phone_score'] = np.where((serv_count['max_serv']==serv_count['serv_count']), 1, 0.9)

#Recency score
vin_recency = RFM_output_clean.drop_duplicates(subset=['vin'], keep='first')[['vin', 'csph_cust_phone_no']].rename(
columns={'csph_cust_phone_no': 'Recent_phone_no'})
Phone_score = pd.merge(serv_count, vin_recency, how='left', on=['unique_cust_grp','vin'])
Phone_score['Ph_rec_score'] = np.where((Phone_score['csph_cust_phone_no']==Phone_score['Recent_phone_no']),
                                        1, 0.9)
RFM_final_analysis = pd.merge(RFM_output, 
                           Phone_score_clean[['unique_cust_grp', 'vin','Ph_rec_score', 'Phone_score']],
                           how='left', on=['unique_cust_grp', 'vin'])
#Assign Referral score
RFM_final_analysis['Referral_tag_score'] = np.where(RFM_final_analysis['Target']==1, 5, 0)

#Calculate Referral score
RFM_output_final['Referral_score'] = RFM_output_final['Final_score']*RFM_output_final['Phone_score']*RFM_output_final['Ph_rec_score']
RFM_output_final['Referral_score'] = RFM_output_final['Referral_score'] + RFM_final_analysis['Referral_tag_score']

RFM_output_final.to_csv('RFM_output.dsv', index=False, encoding='utf-8')